# Puzzle 1: Find the oxygen system using a droid and intcode

### status: 0 --> wall; 1 --> moved 1 step; 2 --> moved 1 step, found oxygen system; movements: north (1), south (2), west (3), east (4)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from copy import deepcopy

## Load input

In [2]:
with open('./input15.txt', 'r') as file:

    software = file.readlines()

#### Convert data to list of integers

In [3]:
software = list(map(int, software[0].split(',')))

## Calculation

In [4]:
class intcode_computer(object):
    
    def __init__(self, data):
        
        self.data = data
        self.data += [0]*100000
        self.inputs = []
        self.i = 0
        self.relative_base = 0
        self.done = False

    def step(self, input):
        self.run = True
        self.out = []
        input = [input]

        while self.run is True:
            add = (5-len(str(self.data[self.i])))*'0'
            self.data[self.i] = '{0}{1}'.format(add, str(self.data[self.i]))

            optcode = self.data[self.i][-2:]
            mode1 = self.data[self.i][-3]
            mode2 = self.data[self.i][-4]
            mode3 = self.data[self.i][-5]

            if mode1 == '0' and optcode != '99':
                param1 = int(self.data[self.data[self.i+1]])
            elif mode1 == '1' and optcode != '99':
                param1 = int(self.data[self.i+1])
            elif mode1 == '2' and optcode != '99':
                param1 = int(self.data[self.data[self.i+1]+self.relative_base])

            if optcode == '01':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                elif mode2 == '1':
                    param2 = int(self.data[self.i+2])
                elif mode2 == '2':
                    param2 = int(self.data[self.data[self.i+2]+self.relative_base])
                    
                if mode3 == '0' or mode3 == '1':
                    self.data[self.data[self.i+3]] = param1 + param2
                elif mode3 == '2':
                    self.data[self.data[self.i+3]+self.relative_base] = param1 + param2
                self.i += 4

            if optcode == '02':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                elif mode2 == '1':
                    param2 = int(self.data[self.i+2])
                elif mode2 == '2':
                    param2 = int(self.data[self.data[self.i+2]+self.relative_base])
                    
                if mode3 == '0' or mode3 == '1':
                    self.data[self.data[self.i+3]] = param1 * param2
                elif mode3 == '2':
                    self.data[self.data[self.i+3]+self.relative_base] = param1 * param2
                self.i += 4

            if optcode == '03':
                if len(input) > 0:
                    if mode1 == '0' or mode1 == '1':
                        self.data[self.data[self.i+1]] = input[0]
                    elif mode1 == '2':
                        self.data[self.data[self.i+1]+self.relative_base] = input[0]
                    del input[0]
                    self.i += 2
                else:
                    self.run = False
                    # print('Waiting for input.')
                    break
                    
            if optcode == '04':
                if mode1 == '0':
                    self.out.append(self.data[self.data[self.i+1]])
                    # print(self.out)
                if mode1 == '1':
                    self.out.append(self.data[self.i+1])
                    # print(self.out)
                elif mode1 == '2':
                    self.out.append(self.data[self.data[self.i+1]+self.relative_base])
                    # print(self.out)
                self.i += 2

            if optcode == '05':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                elif mode2 == '1':
                    param2 = int(self.data[self.i+2])
                elif mode2 == '2':
                    param2 = int(self.data[self.data[self.i+2]+self.relative_base])
                if param1 != 0:
                    self.i = param2
                else:
                    self.i += 3

            if optcode == '06':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                elif mode2 == '1':
                    param2 = int(self.data[self.i+2])
                elif mode2 == '2':
                    param2 = int(self.data[self.data[self.i+2]+self.relative_base])
                    
                if param1 == 0:
                    self.i = param2
                else:
                    self.i += 3

            if optcode == '07':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                elif mode2 == '1':
                    param2 = int(self.data[self.i+2])
                elif mode2 == '2':
                    param2 = int(self.data[self.data[self.i+2]+self.relative_base])
                    
                if param1 < param2:
                    if mode3 == '0' or mode3 == '1':
                        self.data[self.data[self.i+3]] = 1
                    elif mode3 =='2':
                        self.data[self.data[self.i+3]+self.relative_base] = 1
                else:
                    if mode3 == '0' or mode3 == '1':
                        self.data[self.data[self.i+3]] = 0
                    elif mode3 == '2':
                        self.data[self.data[self.i+3]+self.relative_base] = 0
                self.i += 4

            if optcode == '08':
                if mode2 == '0':
                    param2 = int(self.data[self.data[self.i+2]])
                elif mode2 == '1':
                    param2 = int(self.data[self.i+2])
                if mode2 == '2':
                    param2 = int(self.data[self.data[self.i+2]+self.relative_base])

                if param1 == param2:
                    if mode3 == '0' or mode3 == '1':
                        self.data[self.data[self.i+3]] = 1
                    elif mode3 == '2':
                        self.data[self.data[self.i+3]+self.relative_base] = 1
                else:
                    if mode3 == '0' or mode3 == '1':
                        self.data[self.data[self.i+3]] = 0
                    elif mode3 == '2':
                        self.data[self.data[self.i+3]+self.relative_base] = 0
                self.i += 4
            
            if optcode == '09':
                self.relative_base += param1
                self.i += 2

            if optcode == '99':
                self.run = False
                self.done = True
                self.i += 1
                
        return self.out

In [18]:
class repair_droid(object):
    
    def __init__(self):
        self.moveset = {'1': (0, -1), '2': (0, 1), '3': (-1, 0), '4': (1, 0)}
        self.status = {'0': 'w', '1': 'm', '2': 'o'}
        self.control = 0
        self.state = (0, 0)
        self.environment = nx.Graph()
        self.environment.add_node(self.state)
        self.oxygen_state = ()
        
    def step(self, new_status, action):
        current_status = self.status['{0}'.format(new_status[0])]
        if current_status == 'w':
            pass
        elif current_status =='m':
            new_state = (self.state[0] + self.moveset['{0}'.format(action)][0],
                          self.state[1] + self.moveset['{0}'.format(action)][1])
            self.environment.add_edge(self.state, new_state)
            self.state = new_state
        elif current_status == 'o':
            new_state = (self.state[0] + self.moveset['{0}'.format(action)][0],
                          self.state[1] + self.moveset['{0}'.format(action)][1])
            self.environment.add_edge(self.state, new_state)
            self.state = new_state
            self.oxygen_state = new_state
        return current_status
        
    def path_to_oxygen(self):
        path = nx.shortest_path(self.environment, (0, 0), self.oxygen_state)
        path_length = len(path)-1
        return path, path_length
    
    def flood_with_oxygen(self):
        time = nx.eccentricity(self.environment, v=self.oxygen_state)
        return time

In [23]:
computer = intcode_computer(deepcopy(software))
droid = repair_droid()
status = False
while status != 'o':
    random_action = np.random.randint(1, 5)    
    output = computer.step(random_action)
    status = droid.step(output, random_action)
path, path_length = droid.path_to_oxygen()

print('The shortest way to the path requires {0} steps.'.format(path_length))

The shortest way to the path requires 404 steps.


# Puzzle 2: Time required to fill map with oxygen

In [24]:
time = droid.flood_with_oxygen()
print('{0} minutes are required to fill the map with oxygen.'.format(time))

406 minutes are required to fill the map with oxygen.
